In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

from biomarker.data_collection import *
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, ExpSineSquared

import numpy as np

data_collection


In [2]:
EXCLUDE_KEYS = [206, 205, 184, 183, 82, 81, 45]
TEST_KEYS = [218, 217, 216]
C = 1
EPSILON = 0.1
GAMMA = 0.1
LIM = 4
MUL = 500

In [3]:
def create_train_test_matrices(train_keys, test_keys):
    excel = parse_master_file(exclude_keys=EXCLUDE_KEYS)
    test_idxs = list(excel[np.isin(excel['Key'], test_keys)].index)
    excel_test = excel.iloc[test_idxs]
    L_test = get_filename_list(excel_test['Associated data'])
    y_test = excel_test['Output: logK'].values
    y_buck_test = excel_test['Output: logKbucket'].values
    
    excel_train = excel.drop(test_idxs, axis=0)
    # alter the weighting of low logK values:
#     excel_train = duplicate_master(excel_train, 2, 6, how='lt')
    L_train = get_filename_list(excel_train['Associated data'])
    y_train = excel_train['Output: logK'].values
    y_buck_train = excel_train['Output: logKbucket'].values
    
    x1_train, x1_dims = create_x1_matrix(L_train, return_dims=True)
    x4_train, x4_dims = create_x4_matrix(L_train, return_dims=True)
    x5_train, x5_dims = create_x5_matrix(L_train, return_dims=True)
    x6_train, x6_dims = create_x6_matrix(L_train, return_dims=True)
    x7_train, x7_dims = create_x7_matrix(L_train, return_dims=True)
    
    x1_test = create_x1_matrix(L_test, max_dims=x1_dims)
    x4_test = create_x4_matrix(L_test, max_dims=x4_dims)
    x5_test = create_x5_matrix(L_test, max_dims=x5_dims)
    x6_test = create_x6_matrix(L_test, max_dims=x6_dims)
    x7_test = create_x7_matrix(L_test, max_dims=x7_dims)
    
    x10_x17_train = excel_train.iloc[:, 3:-2]
    master_train, master_names = prepare_master(x10_x17_train)
    
    x10_x17_test = excel_test.iloc[:, 3:-2]
    master_test, master_names = prepare_master(x10_x17_test)
    
    X_train = np.hstack((x1_train,x4_train,x5_train,x6_train,x7_train))
    X_test = np.hstack((x1_test, x4_test, x5_test, x6_test, x7_test))
    
    return (X_train, y_train), (X_test, y_test)


def compute_sample_weights(y, lim=LIM, mul=MUL):
    ret = np.ones(y.shape[0])
    for i in range(y.shape[0]):
        v = y[i]
        if v < lim:
            ret[i] = (lim-v)*mul
    return ret
    

In [4]:
excel = parse_master_file(exclude_keys=EXCLUDE_KEYS)
keys = excel['Key']
train_keys = [key for key in keys if key not in TEST_KEYS]
train_set, test_set = create_train_test_matrices(train_keys, TEST_KEYS)

[3, 9]
[3, 9]


In [5]:
X_train = train_set[0]
y_train = train_set[1]
X_test = test_set[0]
y_test = test_set[1]
X_train.shape

(193, 1050)

In [6]:
# Regression predictions
clf = svm.SVR(kernel='rbf', epsilon=0.001, C=10, gamma=1e-6)
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print(prediction, y_test)


[5.80872057 5.80872057 5.84983245] [6.54 1.   1.  ]


In [7]:
# Regression cross-validation
clf = svm.SVR(kernel='rbf', epsilon=EPSILON, C=C, gamma=GAMMA)
cvs = cross_val_score(clf, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print('Mean Cross-Validation Score: {}, Stddev: {}'.format(cvs.mean(), cvs.std()))

# Regression cross-validation
gpr = GaussianProcessRegressor()
cvs = cross_val_score(gpr, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print('Mean Cross-Validation Score: {}, Stddev: {}'.format(cvs.mean(), cvs.std()))

sws = compute_sample_weights(y_train)

clf.fit(X_train,y_train, sample_weight=sws)
prediction=clf.predict(X_test)
print(prediction, y_test)

Mean Cross-Validation Score: -2.0179839166660356, Stddev: 0.553261745266635
Mean Cross-Validation Score: -6.140282041648289, Stddev: 1.1828844258955138
[5.33925877 5.33925877 5.33925877] [6.54 1.   1.  ]


In [8]:
# Parameter grid search
Gammas = [1e-7, 1e-6, 1e-5, 1e-4, 1e-2, 1e-1, 1, 10, 100, 'auto']
Cs = [0.1, 1, 10, 20, 50, 100]
Epsilons = [1e-7, 1e-6, 1e-5, 1e-4, 1e-2, 1e-1, 1, 10, 100]
best_params = [0,0,0]
best_score = -float('inf')
best_prediction = None
for gamma in Gammas:
    for c in Cs:
        for eps in Epsilons:
            clf = svm.SVR(kernel='rbf', epsilon=eps, C=c, gamma=gamma)
            clf.fit(X_train,y_train, sample_weight=sws)
            prediction=clf.predict(X_test)
            diff = y_test[1] - prediction[1]
            print('gamma: {}, c: {}, epsilon: {}, Difference: {}:'.format(gamma, c, eps, diff))
            if diff > best_score:
                best_score = diff
                best_params = [gamma, c, eps]
                best_prediction = prediction[1]

gamma: 1e-07, c: 0.1, epsilon: 1e-07, Difference: -1.902714144282767:
gamma: 1e-07, c: 0.1, epsilon: 1e-06, Difference: -1.9027143457712925:
gamma: 1e-07, c: 0.1, epsilon: 1e-05, Difference: -1.9027163606565463:
gamma: 1e-07, c: 0.1, epsilon: 0.0001, Difference: -1.9027622326129654:
gamma: 1e-07, c: 0.1, epsilon: 0.01, Difference: -1.905271663372357:
gamma: 1e-07, c: 0.1, epsilon: 0.1, Difference: -1.914419109417267:
gamma: 1e-07, c: 0.1, epsilon: 1, Difference: -2.2591591078333866:
gamma: 1e-07, c: 0.1, epsilon: 10, Difference: -7.35:
gamma: 1e-07, c: 0.1, epsilon: 100, Difference: -7.350000000000001:
gamma: 1e-07, c: 1, epsilon: 1e-07, Difference: -3.353293822744842:
gamma: 1e-07, c: 1, epsilon: 1e-06, Difference: -3.3532944453618736:
gamma: 1e-07, c: 1, epsilon: 1e-05, Difference: -3.3533006715322546:
gamma: 1e-07, c: 1, epsilon: 0.0001, Difference: -3.35336273740187:
gamma: 1e-07, c: 1, epsilon: 0.01, Difference: -3.359528638092203:
gamma: 1e-07, c: 1, epsilon: 0.1, Difference: -3.

gamma: 1e-05, c: 1, epsilon: 1, Difference: -4.53689062140438:
gamma: 1e-05, c: 1, epsilon: 10, Difference: -7.35:
gamma: 1e-05, c: 1, epsilon: 100, Difference: -7.350000000000001:
gamma: 1e-05, c: 10, epsilon: 1e-07, Difference: -5.185217605190014:
gamma: 1e-05, c: 10, epsilon: 1e-06, Difference: -5.185217659915591:
gamma: 1e-05, c: 10, epsilon: 1e-05, Difference: -5.185218440006872:
gamma: 1e-05, c: 10, epsilon: 0.0001, Difference: -5.185224122806213:
gamma: 1e-05, c: 10, epsilon: 0.01, Difference: -5.185828847329434:
gamma: 1e-05, c: 10, epsilon: 0.1, Difference: -5.191016909626541:
gamma: 1e-05, c: 10, epsilon: 1, Difference: -5.275041392510165:
gamma: 1e-05, c: 10, epsilon: 10, Difference: -7.35:
gamma: 1e-05, c: 10, epsilon: 100, Difference: -7.350000000000001:
gamma: 1e-05, c: 20, epsilon: 1e-07, Difference: -5.188005543251984:
gamma: 1e-05, c: 20, epsilon: 1e-06, Difference: -5.188005609405878:
gamma: 1e-05, c: 20, epsilon: 1e-05, Difference: -5.1880060760242666:
gamma: 1e-05, 

gamma: 0.01, c: 20, epsilon: 1, Difference: -5.3698611739059:
gamma: 0.01, c: 20, epsilon: 10, Difference: -7.35:
gamma: 0.01, c: 20, epsilon: 100, Difference: -7.350000000000001:
gamma: 0.01, c: 50, epsilon: 1e-07, Difference: -5.266172274409757:
gamma: 0.01, c: 50, epsilon: 1e-06, Difference: -5.266172379640689:
gamma: 0.01, c: 50, epsilon: 1e-05, Difference: -5.266173427219974:
gamma: 0.01, c: 50, epsilon: 0.0001, Difference: -5.266183959996859:
gamma: 0.01, c: 50, epsilon: 0.01, Difference: -5.267136895401208:
gamma: 0.01, c: 50, epsilon: 0.1, Difference: -5.272537636087182:
gamma: 0.01, c: 50, epsilon: 1, Difference: -5.369861173905898:
gamma: 0.01, c: 50, epsilon: 10, Difference: -7.35:
gamma: 0.01, c: 50, epsilon: 100, Difference: -7.350000000000001:
gamma: 0.01, c: 100, epsilon: 1e-07, Difference: -5.266172274409705:
gamma: 0.01, c: 100, epsilon: 1e-06, Difference: -5.266172379640725:
gamma: 0.01, c: 100, epsilon: 1e-05, Difference: -5.266173427220073:
gamma: 0.01, c: 100, epsi

gamma: 10, c: 0.1, epsilon: 1e-07, Difference: -2.1996154461538464:
gamma: 10, c: 0.1, epsilon: 1e-06, Difference: -2.1996160000000007:
gamma: 10, c: 0.1, epsilon: 1e-05, Difference: -2.1996215384615385:
gamma: 10, c: 0.1, epsilon: 0.0001, Difference: -2.199676923076922:
gamma: 10, c: 0.1, epsilon: 0.01, Difference: -2.2057692307692296:
gamma: 10, c: 0.1, epsilon: 0.1, Difference: -2.2600000000000007:
gamma: 10, c: 0.1, epsilon: 1, Difference: -2.8572727272727287:
gamma: 10, c: 0.1, epsilon: 10, Difference: -7.35:
gamma: 10, c: 0.1, epsilon: 100, Difference: -7.350000000000001:
gamma: 10, c: 1, epsilon: 1e-07, Difference: -4.316081105405406:
gamma: 10, c: 1, epsilon: 1e-06, Difference: -4.316081324324324:
gamma: 10, c: 1, epsilon: 1e-05, Difference: -4.3160835135135125:
gamma: 10, c: 1, epsilon: 0.0001, Difference: -4.316105405405401:
gamma: 10, c: 1, epsilon: 0.01, Difference: -4.318378378378379:
gamma: 10, c: 1, epsilon: 0.1, Difference: -4.339258765243908:
gamma: 10, c: 1, epsilon: 

gamma: auto, c: 10, epsilon: 0.01, Difference: -5.227007259253351:
gamma: auto, c: 10, epsilon: 0.1, Difference: -5.231618066206187:
gamma: auto, c: 10, epsilon: 1, Difference: -5.3145460763331815:
gamma: auto, c: 10, epsilon: 10, Difference: -7.35:
gamma: auto, c: 10, epsilon: 100, Difference: -7.350000000000001:
gamma: auto, c: 20, epsilon: 1e-07, Difference: -5.22622696607005:
gamma: auto, c: 20, epsilon: 1e-06, Difference: -5.226233032881288:
gamma: auto, c: 20, epsilon: 1e-05, Difference: -5.226231824831922:
gamma: auto, c: 20, epsilon: 0.0001, Difference: -5.226235817818136:
gamma: auto, c: 20, epsilon: 0.01, Difference: -5.227007259938055:
gamma: auto, c: 20, epsilon: 0.1, Difference: -5.231618163766122:
gamma: auto, c: 20, epsilon: 1, Difference: -5.3145460763331815:
gamma: auto, c: 20, epsilon: 10, Difference: -7.35:
gamma: auto, c: 20, epsilon: 100, Difference: -7.350000000000001:
gamma: auto, c: 50, epsilon: 1e-07, Difference: -5.226227550524507:
gamma: auto, c: 50, epsilon:

In [9]:
best_score

-1.902714144282767

In [10]:
best_params

[1e-07, 0.1, 1e-07]

In [11]:
best_prediction

2.902714144282767

In [12]:
clf = svm.SVR(kernel='rbf', epsilon=best_params[2], C=best_params[1], gamma=best_params[0])
clf.fit(X_train[:100], y_train[:100])
prediction = clf.predict(X_train[100:])
print(prediction)
print(prediction- y_train[100:])

[6.54260756 6.61163071 6.63773858 6.5089014  6.56587978 6.66240872
 6.65698546 6.61698393 6.65103952 6.85873501 6.63787253 6.51956212
 6.85871553 6.53981547 6.65543966 6.43745643 6.67430135 6.65560827
 6.68395018 6.69843814 6.84759308 6.54209878 6.54210815 6.42211464
 6.42155202 6.42573191 6.42670978 6.4213629  6.42022696 6.42152856
 6.4215229  6.4215799  6.42153003 6.42152792 6.42152928 6.42152949
 6.55487185 6.55487185 6.55487185 6.6342712  6.6342712  6.52074085
 6.52074085 6.56831867 6.56831867 6.54015189 6.54471517 6.55440158
 6.55435787 6.45489387 6.35015818 6.35015818 6.35015818 6.52067021
 6.52067021 6.52067021 6.4221045  6.4221045  6.39283476 6.39283476
 6.42203589 6.42203589 6.36579682 6.36579682 6.50847784 6.59765644
 6.66992901 6.5691293  6.61881263 6.40108071 6.44790017 6.42152949
 6.25803479 6.75711897 6.81001495 6.71086113 6.42152949 6.42152949
 6.42152949 6.42825251 6.42574632 6.42277704 6.37476865 6.56591984
 6.40540286 6.29173463 6.29326369 6.6540656  6.25518631 6.6028

In [13]:
# Parameter grid search
Gammas = [1e-7, 1e-6, 1e-5, 1e-4, 1e-2, 1e-1, 1, 10, 100, 'auto']
Cs = [0.1, 1, 10, 20, 50, 100]
Epsilons = [1e-7, 1e-6, 1e-5, 1e-4, 1e-2, 1e-1, 1, 10, 100]
best_params = [0,0,0]
best_score = -float('inf')
best_cvs = None
for gamma in Gammas:
    for c in Cs:
        for eps in Epsilons:
            clf = svm.SVR(kernel='rbf', epsilon=eps, C=c, gamma=gamma)
            sws = compute_sample_weights(y_train)
            cvs = cross_val_score(clf, X_train, y_train, cv=5, scoring='neg_mean_absolute_error', fit_params={'sample_weight':sws})
            print('gamma: {}, c: {}, epsilon: {}, Mean Cross-Validation Score: {}, Stddev: {}'.format(gamma, c, eps, cvs.mean(), cvs.std()))
            if cvs.mean() > best_score:
                best_score = cvs.mean()
                best_cvs = cvs
                best_params = [gamma, c, eps]

gamma: 1e-07, c: 0.1, epsilon: 1e-07, Mean Cross-Validation Score: -3.091499014715901, Stddev: 0.8818800210965277
gamma: 1e-07, c: 0.1, epsilon: 1e-06, Mean Cross-Validation Score: -3.091500804061706, Stddev: 0.8818791843692287
gamma: 1e-07, c: 0.1, epsilon: 1e-05, Mean Cross-Validation Score: -3.091498935942615, Stddev: 0.8818788689857637
gamma: 1e-07, c: 0.1, epsilon: 0.0001, Mean Cross-Validation Score: -3.091475146175267, Stddev: 0.8818876029267204
gamma: 1e-07, c: 0.1, epsilon: 0.01, Mean Cross-Validation Score: -3.08824560834008, Stddev: 0.8798439448245049
gamma: 1e-07, c: 0.1, epsilon: 0.1, Mean Cross-Validation Score: -3.065642831740823, Stddev: 0.8630870474106892
gamma: 1e-07, c: 0.1, epsilon: 1, Mean Cross-Validation Score: -2.74972398383577, Stddev: 0.7726718228566207
gamma: 1e-07, c: 0.1, epsilon: 10, Mean Cross-Validation Score: -2.8308002699055326, Stddev: 0.658509084950861
gamma: 1e-07, c: 0.1, epsilon: 100, Mean Cross-Validation Score: -2.830800269905534, Stddev: 0.6585

gamma: 1e-06, c: 10, epsilon: 1e-05, Mean Cross-Validation Score: -1.8019646335825068, Stddev: 0.39625887715890834
gamma: 1e-06, c: 10, epsilon: 0.0001, Mean Cross-Validation Score: -1.8019754261232694, Stddev: 0.39624875477110194
gamma: 1e-06, c: 10, epsilon: 0.01, Mean Cross-Validation Score: -1.802158682696432, Stddev: 0.39622469151164685
gamma: 1e-06, c: 10, epsilon: 0.1, Mean Cross-Validation Score: -1.803018730507885, Stddev: 0.3970826288710059
gamma: 1e-06, c: 10, epsilon: 1, Mean Cross-Validation Score: -1.8511194004285976, Stddev: 0.45299422889298685
gamma: 1e-06, c: 10, epsilon: 10, Mean Cross-Validation Score: -2.8308002699055326, Stddev: 0.658509084950861
gamma: 1e-06, c: 10, epsilon: 100, Mean Cross-Validation Score: -2.830800269905534, Stddev: 0.6585090849508614
gamma: 1e-06, c: 20, epsilon: 1e-07, Mean Cross-Validation Score: -1.79790423588924, Stddev: 0.3975542092964946
gamma: 1e-06, c: 20, epsilon: 1e-06, Mean Cross-Validation Score: -1.7979043260562524, Stddev: 0.3975

gamma: 1e-05, c: 50, epsilon: 0.01, Mean Cross-Validation Score: -1.9402620947452707, Stddev: 0.496547330239729
gamma: 1e-05, c: 50, epsilon: 0.1, Mean Cross-Validation Score: -1.9342325996073697, Stddev: 0.499805193094758
gamma: 1e-05, c: 50, epsilon: 1, Mean Cross-Validation Score: -1.9481060755787758, Stddev: 0.5639473969294261
gamma: 1e-05, c: 50, epsilon: 10, Mean Cross-Validation Score: -2.8308002699055326, Stddev: 0.658509084950861
gamma: 1e-05, c: 50, epsilon: 100, Mean Cross-Validation Score: -2.830800269905534, Stddev: 0.6585090849508614
gamma: 1e-05, c: 100, epsilon: 1e-07, Mean Cross-Validation Score: -1.9374105961285957, Stddev: 0.4988606500099055
gamma: 1e-05, c: 100, epsilon: 1e-06, Mean Cross-Validation Score: -1.9374088735621204, Stddev: 0.49884741411169403
gamma: 1e-05, c: 100, epsilon: 1e-05, Mean Cross-Validation Score: -1.9374187598333428, Stddev: 0.49885172814667805
gamma: 1e-05, c: 100, epsilon: 0.0001, Mean Cross-Validation Score: -1.9374212249104545, Stddev: 0.

gamma: 0.01, c: 0.1, epsilon: 1, Mean Cross-Validation Score: -2.80102225793335, Stddev: 0.8079278142715287
gamma: 0.01, c: 0.1, epsilon: 10, Mean Cross-Validation Score: -2.8308002699055326, Stddev: 0.658509084950861
gamma: 0.01, c: 0.1, epsilon: 100, Mean Cross-Validation Score: -2.830800269905534, Stddev: 0.6585090849508614
gamma: 0.01, c: 1, epsilon: 1e-07, Mean Cross-Validation Score: -2.1272547984767156, Stddev: 0.5554749355597187
gamma: 0.01, c: 1, epsilon: 1e-06, Mean Cross-Validation Score: -2.127254713672174, Stddev: 0.5554749203821012
gamma: 0.01, c: 1, epsilon: 1e-05, Mean Cross-Validation Score: -2.1272538709408977, Stddev: 0.5554747890323014
gamma: 0.01, c: 1, epsilon: 0.0001, Mean Cross-Validation Score: -2.127245267409543, Stddev: 0.5554733985842895
gamma: 0.01, c: 1, epsilon: 0.01, Mean Cross-Validation Score: -2.126325913918289, Stddev: 0.5553038897125615
gamma: 0.01, c: 1, epsilon: 0.1, Mean Cross-Validation Score: -2.1165640734800286, Stddev: 0.5534584879694148
gamm

gamma: 0.1, c: 20, epsilon: 1e-06, Mean Cross-Validation Score: -2.0529921846326564, Stddev: 0.6080276427156904
gamma: 0.1, c: 20, epsilon: 1e-05, Mean Cross-Validation Score: -2.0529899671673815, Stddev: 0.6080244530787314
gamma: 0.1, c: 20, epsilon: 0.0001, Mean Cross-Validation Score: -2.052992470146226, Stddev: 0.6080240459773472
gamma: 0.1, c: 20, epsilon: 0.01, Mean Cross-Validation Score: -2.0532697235886843, Stddev: 0.6079980905326108
gamma: 0.1, c: 20, epsilon: 0.1, Mean Cross-Validation Score: -2.0554141018537835, Stddev: 0.6080031221228672
gamma: 0.1, c: 20, epsilon: 1, Mean Cross-Validation Score: -2.100375622220194, Stddev: 0.6326294125007502
gamma: 0.1, c: 20, epsilon: 10, Mean Cross-Validation Score: -2.8308002699055326, Stddev: 0.658509084950861
gamma: 0.1, c: 20, epsilon: 100, Mean Cross-Validation Score: -2.830800269905534, Stddev: 0.6585090849508614
gamma: 0.1, c: 50, epsilon: 1e-07, Mean Cross-Validation Score: -2.0529896916752386, Stddev: 0.608024497954717
gamma: 0

gamma: 1, c: 100, epsilon: 0.1, Mean Cross-Validation Score: -2.0554141018537715, Stddev: 0.6080031221228683
gamma: 1, c: 100, epsilon: 1, Mean Cross-Validation Score: -2.1003776781412467, Stddev: 0.6326283845714448
gamma: 1, c: 100, epsilon: 10, Mean Cross-Validation Score: -2.8308002699055326, Stddev: 0.658509084950861
gamma: 1, c: 100, epsilon: 100, Mean Cross-Validation Score: -2.830800269905534, Stddev: 0.6585090849508614
gamma: 10, c: 0.1, epsilon: 1e-07, Mean Cross-Validation Score: -3.2659121619269795, Stddev: 0.9703942929504257
gamma: 10, c: 0.1, epsilon: 1e-06, Mean Cross-Validation Score: -3.2659117705065723, Stddev: 0.9703941644947351
gamma: 10, c: 0.1, epsilon: 1e-05, Mean Cross-Validation Score: -3.2659078563025075, Stddev: 0.9703928799388342
gamma: 10, c: 0.1, epsilon: 0.0001, Mean Cross-Validation Score: -3.2658687142618583, Stddev: 0.970380034479812
gamma: 10, c: 0.1, epsilon: 0.01, Mean Cross-Validation Score: -3.261467845911478, Stddev: 0.9688995579851065
gamma: 10, 

gamma: 100, c: 10, epsilon: 1e-07, Mean Cross-Validation Score: -2.05298969183971, Stddev: 0.6080244978606748
gamma: 100, c: 10, epsilon: 1e-06, Mean Cross-Validation Score: -2.0529921846326564, Stddev: 0.6080276427156902
gamma: 100, c: 10, epsilon: 1e-05, Mean Cross-Validation Score: -2.052989967167385, Stddev: 0.608024453078731
gamma: 100, c: 10, epsilon: 0.0001, Mean Cross-Validation Score: -2.052992470146228, Stddev: 0.6080240459773462
gamma: 100, c: 10, epsilon: 0.01, Mean Cross-Validation Score: -2.0532771249044703, Stddev: 0.6080075339258558
gamma: 100, c: 10, epsilon: 0.1, Mean Cross-Validation Score: -2.055414101853783, Stddev: 0.6080031221228681
gamma: 100, c: 10, epsilon: 1, Mean Cross-Validation Score: -2.100377678141247, Stddev: 0.632628384571445
gamma: 100, c: 10, epsilon: 10, Mean Cross-Validation Score: -2.8308002699055326, Stddev: 0.658509084950861
gamma: 100, c: 10, epsilon: 100, Mean Cross-Validation Score: -2.830800269905534, Stddev: 0.6585090849508614
gamma: 100, c

gamma: auto, c: 50, epsilon: 0.01, Mean Cross-Validation Score: -2.0060928450610835, Stddev: 0.5913900324669645
gamma: auto, c: 50, epsilon: 0.1, Mean Cross-Validation Score: -2.0073620559142444, Stddev: 0.5907542077325135
gamma: auto, c: 50, epsilon: 1, Mean Cross-Validation Score: -2.0507989202608097, Stddev: 0.6162101112965479
gamma: auto, c: 50, epsilon: 10, Mean Cross-Validation Score: -2.8308002699055326, Stddev: 0.658509084950861
gamma: auto, c: 50, epsilon: 100, Mean Cross-Validation Score: -2.830800269905534, Stddev: 0.6585090849508614
gamma: auto, c: 100, epsilon: 1e-07, Mean Cross-Validation Score: -2.005893172417449, Stddev: 0.591494296793662
gamma: auto, c: 100, epsilon: 1e-06, Mean Cross-Validation Score: -2.005899619051436, Stddev: 0.5915022208202542
gamma: auto, c: 100, epsilon: 1e-05, Mean Cross-Validation Score: -2.0059000323961116, Stddev: 0.5915016840353341
gamma: auto, c: 100, epsilon: 0.0001, Mean Cross-Validation Score: -2.0058971121693063, Stddev: 0.591494386091

In [14]:
best_score

-1.7979032754213378

In [15]:
best_params

[1e-06, 20, 1e-05]

In [16]:
best_cvs

array([-1.69703614, -1.62508446, -1.35503853, -2.54176499, -1.77059226])